# Calculate the training accuracy from total_test file

In [32]:
import os
import pandas as pd


def total_test_acc(subj):
    total_data = []
    data_dir = r'/mnt/data/Project/DCM/BIDS/sourcedata/sub_{}/Behaviour/total_test'.format(subj)
    tottest_tmp = os.listdir(data_dir)
    tottest_list = []
    for tot in tottest_tmp:
        if '.csv' in tot:
            tottest_list.append(tot)

    for tot in tottest_list:
        data_path = os.path.join(data_dir, tot)
        print(subj,tot)
        if os.path.getsize(data_path) > 8190:
            tot_data = pd.read_csv(data_path)
            if 'apaccuracy' in tot_data.columns:
                if 'dpaccuracy' in tot_data.columns:
                    tot_acc = tot_data.dropna(subset=['apaccuracy','dpaccuracy'],how='all')
                    tot_acc = tot_acc[['姓名','date','apaccuracy','dpaccuracy']]
                    total_data.append(tot_acc)
                else:
                    tot_acc = tot_data.dropna(subset=['apaccuracy'],how='all')
                    tot_acc = tot_acc[['姓名','date','apaccuracy']]
                    total_data.append(tot_acc)
            elif 'dpaccuracy' in tot_data.columns:
                    tot_acc = tot_data.dropna(subset=['dpaccuracy'],how='all')
                    tot_acc = tot_acc[['姓名','date','dpaccuracy']]
                    total_data.append(tot_acc)
            else:
                print("The {} do not have acc info".format(tot)) 
    return total_data


total_data = [] 
subjests = [str(i).zfill(3) for i in range(29,50)]
for subj in subjests:
    sub_data = total_test_acc(subj)
    if isinstance(sub_data, list):
        total_data.extend(sub_data)
total_data = pd.concat(total_data,axis=0)

i = 1
save_path = r'/mnt/data/Project/DCM/BIDS/derivatives/behaviour/result/training/{}_total_data.csv'.format(i)
while os.path.exists(save_path):
    i += 1
    save_path = r'/mnt/data/Project/DCM/BIDS/derivatives/behaviour/result/training/{}_total_data.csv'.format(i)

total_data.to_csv(save_path)

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/Project/DCM/BIDS/sourcedata/sub_029/Behaviour/total_test'

# Calculate the training accuracy from game2 training file

In [1]:
import os
import pandas as pd


def game2trian_acc(subj):
    data_dir = r'/mnt/workdir/DCM/sourcedata/sub_{}/Behaviour/fmri_task-game2-train'.format(subj)
    game2train_tmp = os.listdir(data_dir)
    game2train_list = []
    for g2t in game2train_tmp:
        if '.csv' in g2t:
            game2train_list.append(g2t)
    game2train_list.sort()

    print('-------------',subj,'start--------------')
    print(game2train_list)
    print("The",subj,"have",len(game2train_list),"runs")

    train_data_blocked = []
    for g2t in game2train_list:
        data_path = os.path.join(data_dir, g2t)
        print(subj,':',g2t)

        if os.path.getsize(data_path) > 12190:
            game2data = pd.read_csv(data_path)
            train_data = game2data.dropna(subset=['train_accuracy'])
            data_len = len(train_data)
            block_num = int(data_len/32)
            for i in range(block_num):
                print('block',i+1)
                if i < (block_num-1):
                    block_data = train_data[i*32:(i+1)*32].copy()    
                else:
                    block_data = train_data[i*32:].copy()

                # 假设这个block是攻击力
                block_ap_diff = block_data['ap_diff']
                block_dp_diff = block_data['dp_diff']

                corrAns_ap = [1 if a < 0 else 2 for a in block_ap_diff]
                corrAns_dp = [1 if a < 0 else 2 for a in block_dp_diff]
                corrAns = block_data['correctAns'].to_list()

                if corrAns == corrAns_ap:
                    block_data.loc[:,'dim'] = 'ap'
                elif corrAns == corrAns_dp:
                    block_data.loc[:,'dim'] = 'dp'
                else:
                    print("The code is wrong! ")

                train_data_blocked.append(block_data.iloc[-1,:][['姓名','participant','date','dim','train_accuracy']])

    return train_data_blocked
    

game2train_result = [] 
subjests = [str(i).zfill(3) for i in range(237,250)]
for subj in subjests:
    sub_result = game2trian_acc(subj)
    if isinstance(sub_result, list):
        game2train_result.extend(sub_result)
        
i = 1
save_path = r'/mnt/workdir/DCM/BIDS/derivatives/behaviour/result/training/{}_game2train_result.csv'.format(i)
while os.path.exists(save_path):
    i += 1
    save_path = r'/mnt/workdir/DCM/BIDS/derivatives/behaviour/result/training/{}_game2train_result.csv'.format(i)

game2train_result = pd.DataFrame(game2train_result)  
#game2train_result.to_csv(save_path)
game2train_result

------------- 237 start--------------
['Z91_task-game2train_2023_Feb_07_1710.csv']
The 237 have 1 runs
237 : Z91_task-game2train_2023_Feb_07_1710.csv
block 1
block 2
block 3
block 4
------------- 238 start--------------
['Z28_task-game2train_2023_Feb_11_1602.csv']
The 238 have 1 runs
238 : Z28_task-game2train_2023_Feb_11_1602.csv
block 1
block 2
block 3
block 4
------------- 239 start--------------
['F26_task-game2train_2023_Feb_11_1704.csv', 'F26_task-game2train_2023_Feb_11_1713.csv']
The 239 have 2 runs
239 : F26_task-game2train_2023_Feb_11_1704.csv
block 1
239 : F26_task-game2train_2023_Feb_11_1713.csv
block 1
block 2
block 3
------------- 240 start--------------
['Z3_task-game2train_2023_Feb_11_1749.csv']
The 240 have 1 runs
240 : Z3_task-game2train_2023_Feb_11_1749.csv
block 1
block 2
block 3
block 4
------------- 241 start--------------
['Z17_task-game2train_2023_Feb_13_1324.csv']
The 241 have 1 runs
241 : Z17_task-game2train_2023_Feb_13_1324.csv
block 1
block 2
block 3
block 4
-

,姓名,participant,date,dim,train_accuracy
64,qinshuting,Z91,2023_Feb_07_1710,dp,0.750
128,qinshuting,Z91,2023_Feb_07_1710,ap,0.938
192,qinshuting,Z91,2023_Feb_07_1710,ap,1.000
256,qinshuting,Z91,2023_Feb_07_1710,dp,0.656
64,yandaoming,Z28,2023_Feb_11_1602,dp,0.781
128,yandaoming,Z28,2023_Feb_11_1602,ap,0.969
192,yandaoming,Z28,2023_Feb_11_1602,ap,1.000
256,yandaoming,Z28,2023_Feb_11_1602,dp,0.938
64,niushengcan,F26,2023_Feb_11_1704,dp,0.719
64,niushengcan,F26,2023_Feb_11_1713,dp,0.625
